In [2]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.embeddings import BedrockEmbeddings
import boto3

file_path = "Ronak_Resume_.pdf"
loader = PyPDFLoader(file_path)

bedrock_runtime = boto3.client(
        service_name="bedrock-runtime",
        region_name="us-east-1",
    )

embeddings = BedrockEmbeddings(
        model_id="amazon.titan-embed-text-v1",
        client=bedrock_runtime,
        region_name="us-east-1",
    )

documents = loader.load()

# Step 2: Split into smaller chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(documents)

vectorstore = FAISS.from_documents(docs, embeddings)

# Step 5: Save to local path
vectorstore.save_local("/tmp/faiss_index")

# Load from disk
faiss_index = FAISS.load_local("/tmp/faiss_index", embeddings, allow_dangerous_deserialization=True)



/tmp/ipykernel_5407/2083294004.py:15: LangChainDeprecationWarning: The class `BedrockEmbeddings` was deprecated in LangChain 0.2.11 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-aws package and should be used instead. To use it run `pip install -U :class:`~langchain-aws` and import as `from :class:`~langchain_aws import BedrockEmbeddings``.
  embeddings = BedrockEmbeddings(


In [15]:
from langchain_community.tools import DuckDuckGoSearchRun

# search = DuckDuckGoSearchRun()
search_tool = DuckDuckGoSearchRun()

# search.invoke("who is jyotirmayee parida who works in operisoft?")

In [19]:
from langchain_community.chat_models import BedrockChat
from langchain_aws import BedrockLLM
llm = BedrockChat(
    model_id="anthropic.claude-3-haiku-20240307-v1:0",
    client=bedrock_runtime,
    region_name="us-east-1",
    model_kwargs={"temperature": 0.0},
)

# llm = BedrockLLM(model_id="anthropic.claude-v2", region_name = "us-east-1")
response = llm.invoke("who is jyotirmayee parida who works in operisoft?")
print(response)

content='I do not have any specific information about a person named Jyotirmayee Parida who works at Operisoft. Operisoft appears to be a software company, but without more context, I cannot provide any verified details about this individual or their role at the company. My knowledge is limited, and I do not have comprehensive information about specific employees of particular organizations.' additional_kwargs={} response_metadata={'model_id': 'anthropic.claude-3-haiku-20240307-v1:0', 'usage': {'prompt_tokens': 24, 'completion_tokens': 83, 'total_tokens': 107}} id='run-9164f7a5-56d6-4574-afd2-5c7791cbf3f6-0'


In [20]:
tools = [search_tool]

# Tool binding
llm_with_tools = llm.bind_tools(tools)

#Tool calling
result = llm_with_tools.invoke("Hello world!")
result
result.content

NotImplementedError: 

In [7]:
from langchain_aws import ChatBedrock
from langchain_community.tools import DuckDuckGoSearchRun

# Initialize the Bedrock chat model with Claude 3 Haiku
llm = ChatBedrock(
    model_id="anthropic.claude-3-haiku-20240307-v1:0",
    region_name="us-east-1",
    model_kwargs={"temperature": 0.0},
)

# Initialize the DuckDuckGo search tool
search_tool = DuckDuckGoSearchRun()
tools = [search_tool]

# Bind the tools to the Bedrock chat model
llm_with_tools = llm.bind_tools(tools)

# Invoke the model with a prompt that can use the tool
result = llm_with_tools.invoke("who is jyotirmayee parida who works in operisoft?")

# Pseudocode for handling tool calls in LangChain

response = llm_with_tools.invoke("Hello world! Can you search for the latest AI news?")

if response.tool_calls:
    # Step 1: Execute the tool call(s)
    tool_outputs = []
    for tool_call in response.tool_calls:
        if tool_call["name"] == "duckduckgo_search":
            query = tool_call["args"]["query"]
            # Actually run the search tool
            search_result = search_tool.run(query)
            tool_outputs.append({
                "tool_call_id": tool_call["id"],
                "output": search_result
            })

    # Step 2: Pass tool outputs back to the model
    final_response = llm_with_tools.invoke(
        "Hello world! Can you search for the latest AI news?",
        tool_outputs=tool_outputs
    )
    print(final_response.content)
else:
    # No tool use, print the model's direct output
    print(response.content)


Error raised by bedrock service
Traceback (most recent call last):
  File "/home/ubuntu/jyoti-env/lib/python3.12/site-packages/langchain_aws/llms/bedrock.py", line 935, in _prepare_input_and_invoke
    response = self.client.invoke_model(**request_options)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ubuntu/jyoti-env/lib/python3.12/site-packages/botocore/client.py", line 570, in _api_call
    return self._make_api_call(operation_name, kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ubuntu/jyoti-env/lib/python3.12/site-packages/botocore/context.py", line 124, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/home/ubuntu/jyoti-env/lib/python3.12/site-packages/botocore/client.py", line 1031, in _make_api_call
    raise error_class(parsed_response, operation_name)
botocore.errorfactory.ValidationException: An error occurred (ValidationException) when calling the InvokeModel operation: Malformed inp

ValidationException: An error occurred (ValidationException) when calling the InvokeModel operation: Malformed input request: #: subject must not be valid against schema {"required":["messages"]}#: extraneous key [tool_outputs] is not permitted, please reformat your input and try again.

In [8]:
from langchain_aws import ChatBedrock
from langchain_community.tools import DuckDuckGoSearchRun

# Step 1: Initialize Bedrock LLM
llm = ChatBedrock(
    model_id="anthropic.claude-3-haiku-20240307-v1:0",
    region_name="us-east-1",
    model_kwargs={"temperature": 0.0}
)

# Step 2: Initialize tools
search_tool = DuckDuckGoSearchRun()
tools = [search_tool]

# Step 3: Bind tools to the LLM
llm_with_tools = llm.bind_tools(tools)

# Step 4: Ask a question that may trigger tool use
query = "Who is Jyotirmayee Parida and what is Operisoft?"

# Step 5: Invoke the model
response = llm_with_tools.invoke(query)

# Step 6: Handle tool calls (if any)
if response.tool_calls:
    tool_outputs = []
    for tool_call in response.tool_calls:
        if tool_call["name"] == "duckduckgo_search":
            # Actually run the search
            tool_result = search_tool.run(tool_call["args"]["query"])
            tool_outputs.append({
                "tool_call_id": tool_call["id"],
                "output": tool_result
            })
    # Step 7: Pass tool outputs back to the model for final answer
    final_response = llm_with_tools.invoke(
        query,
        tool_outputs=tool_outputs
    )
    print(final_response.content)
else:
    # If no tool was called, print the direct answer
    print(response.content)


Error raised by bedrock service
Traceback (most recent call last):
  File "/home/ubuntu/jyoti-env/lib/python3.12/site-packages/langchain_aws/llms/bedrock.py", line 935, in _prepare_input_and_invoke
    response = self.client.invoke_model(**request_options)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ubuntu/jyoti-env/lib/python3.12/site-packages/botocore/client.py", line 570, in _api_call
    return self._make_api_call(operation_name, kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ubuntu/jyoti-env/lib/python3.12/site-packages/botocore/context.py", line 124, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/home/ubuntu/jyoti-env/lib/python3.12/site-packages/botocore/client.py", line 1031, in _make_api_call
    raise error_class(parsed_response, operation_name)
botocore.errorfactory.ValidationException: An error occurred (ValidationException) when calling the InvokeModel operation: Malformed inp

ValidationException: An error occurred (ValidationException) when calling the InvokeModel operation: Malformed input request: #: subject must not be valid against schema {"required":["messages"]}#: extraneous key [tool_outputs] is not permitted, please reformat your input and try again.